In [4]:
import pandas as pd
import pprint
import numpy as np
from neo4j import GraphDatabase, Record
from typing import List, Dict, Tuple
from enum import Enum
pd.set_option('mode.chained_assignment',None)
import numbers
import os
sys.path.append('..')

# import pygraphml
# import neo4j

### PART DATAFRAME

In [5]:
part_df_origin = pd.read_csv("./data/rail_splicing_part.csv")

In [6]:
part_df_origin.head()

,rail,element_code,path,reference,parent
0,1,rar,REPLACE_A1A2371A145D46AA97BA586C90C4A444\E5321...,E53214734200,NaN
1,1,rav,REPLACE_E1819591053E4490B57BFF413D41AB65\E5323...,E53233044200,NaN
2,1,eg,REPLACE_2A6C27350E3E476CA75538AFF81952DB\E5322...,E53225041200,NaN
3,1,ed,REPLACE_2A6C27350E3E476CA75538AFF81952DB\E5322...,E53225041201,NaN
4,1,eq,REPLACE_A9447BE4E2BB4C12BA20BFDFB2A0820B\E5321...,E53214090200,E53214773000


In [7]:
def get_element_name(code:str, el_type:str):
    if el_type == "instance":
        element_corr = {"rar": "front rail", "rav":"rear rail", "eg":"left splice", "ed":"right splice", "eq":"square", "tr":"crossbeam"}
    elif el_type == "part" :
        element_corr = {"rar": "rail", "rav":"rail", "eg":"splice", "ed":"splice", "eq":"square", "tr":"crossbeam"}
    else:
        raise Exception("element type error")
    return element_corr[code]

def get_rail_by_id(id:int):
    rail_y = {1:"Y+1292", 2:"Y+763", 3:"Y+254", 4:"Y-254", 5:"Y-763", 6:"Y-1292"}
    return rail_y[id]

#### PART CLASS DATAFRAME

In [8]:
part_class_df = part_df_origin[['element_code', 'reference']].drop_duplicates('reference')
part_class_df['ename'] = part_class_df.element_code.apply(get_element_name, el_type='part')
del part_class_df['element_code']

In [9]:
part_class_df.head()

,reference,ename
0,E53214734200,rail
1,E53233044200,rail
2,E53225041200,splice
3,E53225041201,splice
4,E53214090200,square


#### PART INSTANCE DATAFRAME

In [10]:
# je recupere les instances
part_instance_df = part_df_origin[['element_code', 'rail', 'reference']]
part_instance_df.rail = part_instance_df.rail.apply(get_rail_by_id)
part_instance_df['ename'] = part_instance_df.element_code.apply(get_element_name, el_type='instance')
del part_instance_df['element_code']
part_instance_df = part_instance_df.groupby(['ename', 'rail']).first()

In [11]:
# je dois extraire les data crossbeam (une seule crossbeam pour tous les rails)
# je recupere la ref
crossbeam_ref = part_instance_df.loc['crossbeam'].iloc[0].reference
# j'extrait les data crossbeam
part_instance_df.drop('crossbeam', inplace=True)

In [12]:
# et je cree une ligne unique avec seulement la crossbeam
part_instance_df.loc[('crossbeam', 'C35'), 'reference'] = crossbeam_ref

In [13]:
part_instance_df.tail()

reference
ename     rail                
square    Y+763   E53215521200
          Y-1292  E53214090200
          Y-254   E53215522200
          Y-763   E53215521201
crossbeam C35     E53214476200

In [18]:
pd.concat([part_instance_df.loc['front rail'], part_instance_df.loc['rear rail']], axis=1)

,reference,reference
rail,,
Y+1292,E53214734200,E53233044200
Y+254,E53215529200,E53233024200
Y+763,E53215518200,E53233046200
Y-1292,E53214735200,E53233043200
Y-254,E53215529201,E53233025200
Y-763,E53215518202,E53233045200


### FASTENER SOURCE DATAFRAME 

In [12]:
fastener_df = pd.read_json("./data/C35_Fasteners_data.json", orient='records')

In [13]:
fastener_df.columns = ['xe', 'ye', 'ze', 'xdir', 'ydir', 'zdir', 'fastener_type',
       'fastener_diameter', 'geometrical_length', 'hole_depth',
       'head_orientation', 'sealant_thickness', 'sealant_standard',
       'temporary_fastener_diameter', 'assembling_condition',
       'local_tolerance', 'head_type', 'parts_to_tight',
       'parts_material_stack', 'parts_thickness_stack',
       'references_item_stack', 'bom_item_stack', 'countersink_angle',
       'synchronized', 'comments', 'countersink_depth', 'id', 'head_side',
       'fastener_concept_part', 'fastener_concept_id', 'attributes',
       'refhfuuid', 'lockuuid', 'remind_note', 'aname', 'parent_name']

In [14]:
# Cast xdir, ydir et zdir en int pour supprimer les valeur avec exp-16
fastener_df.xdir = pd.to_numeric(fastener_df.xdir, downcast='integer')
fastener_df.ydir = pd.to_numeric(fastener_df.ydir, downcast='integer')
fastener_df.zdir = pd.to_numeric(fastener_df.zdir, downcast='integer')

# ajout de 15367 en X pour se caler sur la reference avion
fastener_df.xe = fastener_df.xe + 15367

In [15]:
def getRailPosition(yValue: float) -> str :
    position:str
    rail:str
    v:float

      # get the side (inverted in catia)
    if yValue > 0 : 
        position = 'RIGHT'
        v = yValue
    else:
        position = 'LEFT'
        v = yValue*-1

    if v < 1760 and v > 1710 :
        rail = "1732"
    elif v < 1310 and v > 1260 :
        rail = "1292"
    elif v < 790 and v > 740 :
        rail = "763"
    elif v < 280 and v > 230 :
        rail = "254"

    return 'Y{signe}{position}'.format(signe='+' if yValue < 0 else '-', position=rail)

In [16]:
# get the rail position
fastener_df['rail_position'] = fastener_df.ye.apply(getRailPosition)

# extract the reference
fastener_df['reference'] = fastener_df.aname.str.extract(r'^([^\.]*)')

# replace fastener types
fastener_df.loc[fastener_df.reference.str.match(r'^en6115'), 'fastener_type'] = "Hi-Lite"
fastener_df.loc[fastener_df.reference.str.match(r'^asna2392'), 'fastener_type'] = "Lockbolt"
fastener_df.loc[fastener_df.parts_to_tight.notna(), "parts_to_tight"] = fastener_df.parts_to_tight.str.split(';')
fastener_df.loc[fastener_df.parts_material_stack.notna(),"parts_material_stack"] = fastener_df.parts_material_stack.str.split(';')
fastener_df.loc[fastener_df.parts_thickness_stack.notna(),"parts_thickness_stack"] = fastener_df.parts_thickness_stack.str.replace('mm', "").str.split(';')
fastener_df.loc[fastener_df.references_item_stack.notna(),"references_item_stack"] = fastener_df.references_item_stack.str.split(';')

fastener_df.xe = fastener_df.xe.round(3)
fastener_df.ye = fastener_df.ye.round(3)
fastener_df.ze = fastener_df.ze.round(3)

In [17]:
fastener_df.ydir.drop_duplicates()

0      0
38     1
134   -1
Name: ydir, dtype: int8

In [18]:
fastener_df.iloc[0].to_dict()

{'xe': 15444.666,
 'ye': 1747.0,
 'ze': -505.0,
 'xdir': 0,
 'ydir': 0,
 'zdir': -1,
 'fastener_type': 'Rivet',
 'fastener_diameter': 4.8,
 'geometrical_length': 5.600001976074,
 'hole_depth': 6.000001976074,
 'head_orientation': 0,
 'sealant_thickness': 0.2,
 'sealant_standard': nan,
 'temporary_fastener_diameter': 0,
 'assembling_condition': 'None',
 'local_tolerance': 'A205',
 'head_type': 'COUNTERSUNK HEAD On countersunk part',
 'parts_to_tight': ['REPLACE_80D083C7CC724AD7A70E23A2F3D72DA2',
  'REPLACE_1EDF4A6A74004583A3242E47A709B634',
  'E53225039201.1'],
 'parts_material_stack': ['7010/7050-T7451_(AIMS03-02-022)(ABS5323)',
  '2024-T3_(AIMS03-04-011)(ASNA3012)',
  '?'],
 'parts_thickness_stack': ['1.8', '1.8', '2'],
 'references_item_stack': ['asna2049DXJ048'],
 'bom_item_stack': nan,
 'countersink_angle': 99.9999999998563,
 'synchronized': 'True',
 'comments': nan,
 'countersink_depth': 0,
 'id': 19720191353712,
 'head_side': 'NotStated',
 'fastener_concept_part': nan,
 'fastener

### MARS DATAFRAME 

#### EXTRACT INFO FROM FASTENER SOURCE DATAFRAME

In [19]:
col_to_keep = ['xe', 'ye', 'ze', 'xdir', 'ydir', 'zdir',
               'parts_to_tight','parts_thickness_stack',
               'id','aname','rail_position', 'reference',
              'parts_material_stack']

# j'extrait les fastener dedié au mars (+1292 à -1292)
mars_df = fastener_df[col_to_keep][(fastener_df.rail_position != "Y+1732") & (fastener_df.rail_position != "Y-1732")]

#### INSERT AREA INFORMATIONS

In [20]:
# ajout de la crossbeam side rear ou front
mars_df["crossbeam_side"] = mars_df.xe.apply(lambda xe : "rear" if xe > 15367 else "front")
# ajout de la area web ou flange
mars_df['area'] = mars_df.zdir.apply(lambda zdir: "web" if zdir != -1 else "flange")

# ajout du rail side left ou right
rail_pos = [254, 1292, 763]
for p in rail_pos:
    mars_df.loc[(mars_df.rail_position== 'Y+%d'%(p)) & (mars_df.zdir !=0) & (mars_df.ye *-1 > p), "rail_side"] = "left"
    mars_df.loc[(mars_df.rail_position== 'Y-%d'%(p)) & (mars_df.zdir !=0) & (mars_df.ye < p), "rail_side"] = "left"

mars_df.loc[(mars_df.zdir !=0)&(mars_df.rail_side.isna()), "rail_side"] = "right"

#### INSERT ELEMENT STACK INFORMATIONS

In [21]:
# fonction pour recuperer le stack des element en fonction de la zone du fastener
def get_element_stack(row:pd.Series) -> str:
    
    ptt = {
           "frontweb":["left splice", "front rail", "right splice"],
           "rearweb":["left splice", "rear rail", "right splice"],
           "frontflangeleftinsideexternal": ["left splice", "front rail"],
           "frontflangeleftinsideinternal": ["left splice", "front rail", "square"],
           "frontflangeleftoutsideexternal": ["left splice", "front rail"],
           "frontflangeleftoutsideinternal": ["left splice", "square"],
           "frontflangerightinsideexternal": ["right splice", "front rail"],
           "frontflangerightinsideinternal": ["right splice", "front rail", "square"],
           "frontflangerightoutsideexternal": ["right splice", "front rail"],
           "frontflangerightoutsideinternal": ["right splice", "square"],
           "rearflangeleftinsideexternal": ["left splice", "rear rail"],
           "rearflangeleftinsideinternal": ["left splice", "rear rail", "crossbeam"],
           "rearflangeleftoutsideexternal": ["left splice", "rear rail"],
           "rearflangeleftoutsideinternal": ["left splice", "rear rail", "crossbeam"],
           "rearflangerightinsideexternal": ["right splice", "rear rail"],
           "rearflangerightinsideinternal": ["right splice", "rear rail", "crossbeam"],
           "rearflangerightoutsideexternal": ["right splice", "rear rail"],
           "rearflangerightoutsideinternal": ["right splice", "rear rail", "crossbeam"]
          }
    
    cside = row.crossbeam_side
    area = row.area
    rside = ""
    rpos = ""
    xpos = ""
    
    if area == "flange" :
        rside = row.rail_side
        rpos = "inside" if row.rail_position.find('254') !=-1 else "outside"
        xpos = "internal" if (row.xe > 15320 and row.xe < 15398) else "external"
    
    pattern = "{cside}{area}{rside}{rpos}{xpos}".format(cside=cside, area=area, rside=rside, rpos=rpos, xpos=xpos)
    
    return ','.join(ptt[pattern])

#ajout du stack des elements avec la fonction
mars_df['el_to_tight'] = mars_df[['crossbeam_side', 'area', 'rail_position', 'xe', 'rail_side']].apply(get_element_stack, axis=1)
mars_df['el_to_tight'] = mars_df['el_to_tight'].str.split(',')

In [22]:
# fonction pour remplacer les el_to_tight par les refrence en fonction de la zone
# je vais chercher les ref dans part_df
def replace_reference(row:pd.Series):
    ref_tab = []
    el_stack = row.el_to_tight
    
    for el in el_stack:
        yref = row.rail_position if not el == 'crossbeam' else "C35"
        ref_tab.append(part_instance_df.loc[(el, yref),'reference'])
    
    return ref_tab

# remplacement element par reference et creation ref_to_tight
mars_df['ref_to_tight'] = mars_df[['el_to_tight', 'rail_position']].apply(replace_reference, axis=1)

In [23]:
mars_df.iloc[0].to_dict()

{'xe': 15475.0,
 'ye': 242.975,
 'ze': -551.6,
 'xdir': 0,
 'ydir': 0,
 'zdir': -1,
 'parts_to_tight': ['E53225034200.2',
  'REPLACE_E4B1DB35975442AF8C4CA22C0F2C2943'],
 'parts_thickness_stack': ['1.6', '1.8'],
 'id': 227201910577297,
 'aname': 'asna2392-3-03.15',
 'rail_position': 'Y-254',
 'reference': 'asna2392-3-03',
 'parts_material_stack': ['?', 'Aluminium'],
 'crossbeam_side': 'rear',
 'area': 'flange',
 'rail_side': 'left',
 'el_to_tight': ['left splice', 'rear rail'],
 'ref_to_tight': ['E53225034200', 'E53233025200']}

#### INSERT TEMPORARY FASTENING INFORMATION

Integration de l'information temporary fastening pour les assembly qui sont utilisés pour maintenir les part en place 

In [24]:
# import de la liste des temporary fastening coté gauche
tmp_drilling_df = pd.read_csv("./data/rail_splicing_tmp_drill.csv")

In [25]:
# je vais chercher dans mars_df les information de position des tmp fastening ci dessus
floc_df =  mars_df[['rail_position', 'aname', 'xe', 'ye', 'ze']][mars_df.aname.isin(tmp_drilling_df.assy.array)]

In [26]:
# je defini une serie les coordonne (x,z) des tmp fastening coté gauche - ils sont identique coté droit
coord = floc_df[['xe', 'ze']].apply(tuple, axis=1).drop_duplicates().array

In [27]:
# je defini une colonne temporary_fastening dans mars_df avec la valeur true pour ceux avec les (x,z) obetenu ci dessus
mars_df.loc[fastener_df[['xe', 'ze']].apply(tuple, axis=1).isin(coord), 'temporary_fastening'] = True
# je met tous les autres de la colonne à false
mars_df.temporary_fastening[mars_df.temporary_fastening !=True] = False

In [28]:
mars_df.iloc[0].to_dict()

{'xe': 15475.0,
 'ye': 242.975,
 'ze': -551.6,
 'xdir': 0,
 'ydir': 0,
 'zdir': -1,
 'parts_to_tight': ['E53225034200.2',
  'REPLACE_E4B1DB35975442AF8C4CA22C0F2C2943'],
 'parts_thickness_stack': ['1.6', '1.8'],
 'id': 227201910577297,
 'aname': 'asna2392-3-03.15',
 'rail_position': 'Y-254',
 'reference': 'asna2392-3-03',
 'parts_material_stack': ['?', 'Aluminium'],
 'crossbeam_side': 'rear',
 'area': 'flange',
 'rail_side': 'left',
 'el_to_tight': ['left splice', 'rear rail'],
 'ref_to_tight': ['E53225034200', 'E53233025200'],
 'temporary_fastening': True}

#### INSERT TEMPORARY DRILLING INFORMATION

Les temporary drilling sont pratiquement identique au temporary fastening, j'extrait juste les assembly de la crossbeam car ils sont déjà prépercé

In [29]:
# je recupere les index des fastener crossbeam
# je cree un dataframe qui est une transposition de la liste des element fasten par les assy decris dans mars_df
temp_part_df = pd.DataFrame(mars_df.el_to_tight.array, index=mars_df.index)
# j'identifie les ligne contenant la crossbeam en index 2, je recupere les index
crossbeam_index = temp_part_df[temp_part_df[2] == "crossbeam"].index

In [30]:
# je cree une colonne temporary_drilling dans mars_df = a temporary_fastening 
mars_df["temporary_drilling"] = mars_df.temporary_fastening.copy()
# je met les ligne decrivant les assy de la crossbeam a false
mars_df.loc[crossbeam_index, "temporary_drilling"]=False

In [31]:
mars_df.iloc[0].to_dict()

{'xe': 15475.0,
 'ye': 242.975,
 'ze': -551.6,
 'xdir': 0,
 'ydir': 0,
 'zdir': -1,
 'parts_to_tight': ['E53225034200.2',
  'REPLACE_E4B1DB35975442AF8C4CA22C0F2C2943'],
 'parts_thickness_stack': ['1.6', '1.8'],
 'id': 227201910577297,
 'aname': 'asna2392-3-03.15',
 'rail_position': 'Y-254',
 'reference': 'asna2392-3-03',
 'parts_material_stack': ['?', 'Aluminium'],
 'crossbeam_side': 'rear',
 'area': 'flange',
 'rail_side': 'left',
 'el_to_tight': ['left splice', 'rear rail'],
 'ref_to_tight': ['E53225034200', 'E53233025200'],
 'temporary_fastening': True,
 'temporary_drilling': True}

### FASTENER PART DATAFRAME

#### FASTENER CLASS DATAFRAME

In [32]:
fastener_class_df = pd.concat([mars_df.reference.drop_duplicates(), fastener_df[['fastener_type','fastener_diameter']]], axis=1, join="inner")

In [33]:
fastener_class_df

,reference,fastener_type,fastener_diameter
14,asna2392-3-03,Lockbolt,4.8
38,asna2392-3-04,Lockbolt,4.8
238,asna2392-3-05,Lockbolt,4.8
266,asna2392-3-07,Lockbolt,4.8
282,en6115V3-4,Hi-Lite,4.8
298,en6115V3-5,Hi-Lite,4.8
306,en6115V3-7,Hi-Lite,4.8


#### FASTENER INSTANCE DATAFRAME

In [34]:
fastener_instance_df = mars_df[['id','reference', 'aname']]

In [35]:
fastener_instance_df.head()

,id,reference,aname
14,227201910577297,asna2392-3-03,asna2392-3-03.15
15,227201910577298,asna2392-3-03,asna2392-3-03.16
16,227201910577299,asna2392-3-03,asna2392-3-03.17
17,2272019105749300,asna2392-3-03,asna2392-3-03.18
18,2272019105749301,asna2392-3-03,asna2392-3-03.19


### IMPORT CLASSES TO CREATE NODES

In [141]:
import os
from neomodel import config
bolt_url = os.environ['NEO4J_BOLT_URL'].replace('"', '')
config.DATABASE_URL = bolt_url

from marsmodel.product.fastener import Instance as FastInstance, Class as FastClass, Fastener
from marsmodel.product.part import Instance as PartInstance, Class as PartClass
from marsmodel.process.pattern import Pattern
from marsmodel.product.assembly import Assembly
from marsmodel.product import Product
from neomodel.util import Database
from neomodel.contrib.spatial_properties import NeomodelPoint

### CREATE ASSEMBLY/FASTENER NODES + RELATIONSHIP

#### ASSEMBLY NODES

In [37]:
# assembly node generation function
def generate_assy_node(row:pd.Series) -> Assembly:
    
    origin = NeomodelPoint(x=row.xe, y=row.ye, z=row.ze)
    orient = [row.xdir, row.ydir, row.zdir]
    name = "assy."+row.aname
    uid = str(row.id)
    
    return Assembly(origin=origin,
                    orient=orient,
                    name=name,
                    uid=uid)

In [38]:
# generate the neomodel assembly nodes
assy_col = ["xe", "ye", "ze", "xdir", "ydir", "zdir", "aname", 'id']
assy_nodes_s = mars_df[assy_col].apply(generate_assy_node, axis=1)
assy_nodes_s.name = "assy_node"

In [39]:
assy_nodes_s.index = mars_df.id

#### FASTENER INSTANCE NODES

In [40]:
def generate_ifastener_node(row:pd.Series) -> FastInstance:
    name="fast."+row.aname
    reference=row.reference
    uid=str(row.id)
    
    return FastInstance(name=name, reference=reference, uid=uid)

In [41]:
fastener_instance_nodes_s = fastener_instance_df.apply(generate_ifastener_node, axis=1)
fastener_instance_nodes_s.name = "fastener_instance_node"
fastener_instance_nodes_s.index = fastener_instance_df.id

#### SAVE ASSY AND FASTENER INSTANCE NODES + RELATIONSHIP

In [42]:
ifastener_assy_df = pd.concat([assy_nodes_s, fastener_instance_nodes_s], axis=1)

In [43]:
ifastener_assy_df.index = fastener_instance_df.id

In [44]:
ifastener_assy_df.head()

,assy_node,fastener_instance_node
id,,
227201910577297,"{'uid': '227201910577297', 'name': 'assy.asna2...","{'uid': '227201910577297', 'name': 'fast.asna2..."
227201910577298,"{'uid': '227201910577298', 'name': 'assy.asna2...","{'uid': '227201910577298', 'name': 'fast.asna2..."
227201910577299,"{'uid': '227201910577299', 'name': 'assy.asna2...","{'uid': '227201910577299', 'name': 'fast.asna2..."
2272019105749300,"{'uid': '2272019105749300', 'name': 'assy.asna...","{'uid': '2272019105749300', 'name': 'fast.asna..."
2272019105749301,"{'uid': '2272019105749301', 'name': 'assy.asna...","{'uid': '2272019105749301', 'name': 'fast.asna..."


In [45]:
def save_assifast_nrs(row:pd.Series) -> bool:
    assy = row.assy_node.save()
    fast = row.fastener_instance_node.save()
    return assy.fastener.connect(fast)

In [46]:
relation_result = ifastener_assy_df.apply(save_assifast_nrs, axis=1)

In [47]:
dict(ifastener_assy_df.iloc[0])

{'assy_node': <Assembly: {'uid': '227201910577297', 'name': 'assy.asna2392-3-03.15', 'orient': [0, 0, -1], 'origin': <neomodel.contrib.spatial_properties.NeomodelPoint object at 0x0000028803101DA0>, 'id': 287}>,
 'fastener_instance_node': <Instance: {'uid': '227201910577297', 'name': 'fast.asna2392-3-03.15', 'reference': 'asna2392-3-03', 'id': 288}>}

### PART NODES

#### PART CLASS NODE

In [48]:
def generate_cpart_node(row:pd.Series) -> PartClass:
    reference = row.reference
    uid = row.reference
    name = row.ename
    
    return PartClass(uid=uid, name=name, reference=reference)

In [49]:
part_class_nodes = part_class_df.apply(generate_cpart_node, axis=1)

In [50]:
part_class_nodes.name = "part_class_node"

In [51]:
part_class_df = part_class_df.join(part_class_nodes)

In [52]:
part_class_df.index = part_class_df.reference

In [53]:
part_class_nodes = part_class_df.part_class_node

#### PART INSTANCE NODES

In [54]:
def generate_ipart_node(row:pd.Series) -> PartInstance:
    name, yrail = row.name
    
    if yrail.find('+') != -1 or yrail.find('-') != -1:
        side = "L" if yrail[1] =="+" else "R"
        uid = row.reference+side+yrail[2:]
    else:
        uid= row.reference+yrail
    
    name = "{name} {yrail}".format(name=name, yrail=yrail)
    reference = row.reference
    
    return PartInstance(uid=uid, reference=reference, name=name)

In [55]:
part_instance_nodes = part_instance_df.apply(generate_ipart_node, axis=1)

In [56]:
part_instance_nodes.index = part_instance_nodes.apply(lambda node : node.uid)

In [57]:
part_instance_nodes.head(3)

E53214734200L1292    {'uid': 'E53214734200L1292', 'name': 'front ra...
E53215529200L254     {'uid': 'E53215529200L254', 'name': 'front rai...
E53215518200L763     {'uid': 'E53215518200L763', 'name': 'front rai...
dtype: object

#### SAVE PART NODES + RELATIONSHIP

In [58]:
# save part class nodes
for part in part_class_df.part_class_node:
    part.save()

In [59]:
# save part instance nodes + create relationship with class nodes
for instance in part_instance_nodes:
    instance.save() # save instance
    mother_class_node = part_class_nodes.loc[instance.reference] # identify class
    instance.mother_class.connect(mother_class_node) # connect to class

### FASTENER NODES

#### FASTENER CLASS NODES

In [60]:
def generate_cfastener_node(row:pd.Series) -> FastClass:
    uid = row.reference
    reference = row.reference
    _type = row.fastener_type
    diameter = row.fastener_diameter
    name=row.fastener_type
    
    return FastClass(uid=uid, name=name, reference=reference, type=_type, diameter=diameter)

In [61]:
fastener_class_nodes = fastener_class_df.apply(generate_cfastener_node, axis=1)

In [62]:
fastener_class_nodes.index = fastener_class_df.reference

In [63]:
fastener_class_nodes

reference
asna2392-3-03    {'uid': 'asna2392-3-03', 'name': 'Lockbolt', '...
asna2392-3-04    {'uid': 'asna2392-3-04', 'name': 'Lockbolt', '...
asna2392-3-05    {'uid': 'asna2392-3-05', 'name': 'Lockbolt', '...
asna2392-3-07    {'uid': 'asna2392-3-07', 'name': 'Lockbolt', '...
en6115V3-4       {'uid': 'en6115V3-4', 'name': 'Hi-Lite', 'refe...
en6115V3-5       {'uid': 'en6115V3-5', 'name': 'Hi-Lite', 'refe...
en6115V3-7       {'uid': 'en6115V3-7', 'name': 'Hi-Lite', 'refe...
dtype: object

#### SAVE FASTENER CLASS NODES

In [64]:
for node in fastener_class_nodes:
    node.save()

#### CONNECT FASTENER INSTANCE AND CLASS NODES

In [65]:
for instance in fastener_instance_nodes_s:
    mother_class = fastener_class_nodes.loc[instance.reference]
    instance.mother_class.connect(mother_class)

### RELATIONSHIP ASSEMBLY -> PART INSTANCE

#### DEFINE AN ASSEMBLY STACK DATAFRAME

In [66]:
# define a function to generate the stack from mars_df dataframe
def generate_stack(row:pd.Series):
    stack_list = []
    
    thickness = row.parts_thickness_stack
    materials = row.parts_material_stack
    references = row.ref_to_tight
    
    side = "L" if row.rail_position[1] == "+" else "R"
    rail = side+row.rail_position[2:]
    
    for index, element in enumerate(row.el_to_tight):
        ref = references[index]+rail if element != "crossbeam" else references[index]+'C35'
        stack_list.append((ref, thickness[index], materials[index]))
    
    return stack_list


In [67]:
# define columns to keep from mars_df 
ass_partinstance_col = ['id', 'parts_thickness_stack','parts_material_stack', 'el_to_tight', 'ref_to_tight', 'rail_position']
# create a serie describing the stacks
ass_stack_s = mars_df[ass_partinstance_col].apply(generate_stack, axis=1)

In [68]:
# visualise the result
ass_stack_s.head(3)

14    [(E53225034200R254, 1.6, ?), (E53233025200R254...
15    [(E53225034200R254, 1.6, ?), (E53233025200R254...
16    [(E53225034200R254, 1.6, ?), (E53233025200R254...
dtype: object

In [69]:
# create a dataframe containing stacks from serie with a columns for each item in list
ass_stack_df= pd.DataFrame(ass_stack_s.array, index=ass_stack_s.index)
# rename the columns => stack+index
ass_stack_df.columns = ["stack"+str(c) for c in ass_stack_df.columns]
# join with mars_df.id to get the assembly id
ass_stack_df = ass_stack_df.join(mars_df.id)

In [70]:
ass_stack_df.head(3)

,stack0,stack1,stack2,id
14,"(E53225034200R254, 1.6, ?)","(E53233025200R254, 1.8, Aluminium)",None,227201910577297
15,"(E53225034200R254, 1.6, ?)","(E53233025200R254, 1.8, Aluminium)",None,227201910577298
16,"(E53225034200R254, 1.6, ?)","(E53233025200R254, 1.8, Aluminium)",None,227201910577299


In [71]:
#wide the dataframe to get a line for each stack
ass_stack_wide = pd.wide_to_long(ass_stack_df, stubnames="stack", i='id', j='index')
# drop the row where no stack2
ass_stack_wide.dropna(axis=0, subset=['stack'], inplace=True)

In [72]:
ass_stack_wide.head()

,,stack
id,index,
227201910577297,0,"(E53225034200R254, 1.6, ?)"
227201910577298,0,"(E53225034200R254, 1.6, ?)"
227201910577299,0,"(E53225034200R254, 1.6, ?)"
2272019105749300,0,"(E53225034201R254, 1.6, ?)"
2272019105749301,0,"(E53225034201R254, 1.6, ?)"


#### DEFINE THE RELATIONSHIP ASSEMBLY -> PART

In [73]:
# code to connect assembly nodes to part instances
# iterate in the list of stacks created below
for stack_id, stack in ass_stack_wide.iterrows():
    
    assy_id, index = stack_id
    stack_def = stack.stack
    
    # define the properties of relation
    rel_properties = dict(stackMaterial=stack_def[2],
                          stackThickness=float(stack_def[1]),
                          stackIndex=index)
    
    # select the assembly node
    assy = assy_nodes_s.loc[assy_id]
    # select the part instance node
    ipart = part_instance_nodes.loc[stack_def[0]]
    
    # create the connection between assy and part instance
    assy.assemble.connect(ipart, rel_properties)
    

### CREATE PATTERN NODE + RELATIONSHIP

In [142]:
# create and save web pattern
web = Pattern(uid='web', name='web').save()

In [168]:
# connect all web assembly to web pattern
for node in assy_nodes_s[assy_nodes_s.apply(lambda node: node.orient[1]!=0)]:
    node.pattern.connect(web)

In [169]:
# create and save flange 
flange = Pattern(uid='flange', name='flange').save()

In [173]:
# connect all flange assembly to flange pattern
for node in assy_nodes_s[assy_nodes_s.apply(lambda node: node.orient[2]!=0)]:
    node.pattern.connect(flange)

### OPERATION NODE

In [ ]:
mars_df[['id','temporary_fastening', 'temporary_drilling']]